In [ ]:
#### modules included in the hklconv_new directory
import os
import sys
sys.path.append("/home/jg2364/indexing/hklconv_new/")
os.chdir("/home/jg2364/indexing/hklconv_new/")
import pyFAI

# New Section

In [ ]:
### functions 
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

## determine polar coordinate from XY
def XYtoPOLAZ(X,Y):
    return newpol[X,Y],newaz[X,Y]

### calculate the UB matrix from euler angles and crystal properties, originall written by J. Ruff
def calcUB(eu1,eu2,eu3,a,b,c,alpha,beta,gamma):
    a1=a
    a2=b
    a3=c
    alpha1=alpha*np.pi/180.0
    alpha2=beta*np.pi/180.0
    alpha3=gamma*np.pi/180.0
    Vt = (1.0 - (np.cos(alpha1)*np.cos(alpha1)) - (np.cos(alpha2)*np.cos(alpha2)) - (np.cos(alpha3)*np.cos(alpha3)))
    Vt = Vt + (2.0*np.cos(alpha1)*np.cos(alpha2)*np.cos(alpha3))
    V = (Vt**0.5)*a1*a2*a3
    b1=2.0*np.pi*a2*a3*np.sin(alpha1)/V
    b2=2.0*np.pi*a3*a1*np.sin(alpha2)/V
    b3=2.0*np.pi*a1*a2*np.sin(alpha3)/V
    betanum=((np.cos(alpha2)*np.cos(alpha3)) - np.cos(alpha1))
    betaden=(np.sin(alpha2)*np.sin(alpha3))
    beta1=np.arccos(betanum/betaden)
    betanum=((np.cos(alpha1)*np.cos(alpha3)) - np.cos(alpha2))
    betaden=(np.sin(alpha1)*np.sin(alpha3))
    beta2=np.arccos(betanum/betaden)
    betanum=((np.cos(alpha1)*np.cos(alpha2)) - np.cos(alpha3))
    betaden=(np.sin(alpha1)*np.sin(alpha2))
    beta3=np.arccos(betanum/betaden)
    UB=np.zeros(9)
    UB[0]=b1
    UB[1]=b2*np.cos(beta3)
    UB[2]=b3*np.cos(beta2)
    UB[3]=0.0
    UB[4]=-b2*np.sin(beta3)
    UB[5]=b3*np.sin(beta2)*np.cos(alpha1)
    UB[6]=0.0
    UB[7]=0.0
    UB[8]=-b3
    r = R.from_euler('zxz', [eu1, eu2, eu3])
    UBR=np.matmul(r.as_matrix(),UB.reshape(3,3))
    return UBR

### calculate chi^2 of a UB matrix
def chisq_peaklist(peaklist,wl,UBR,eta,mu,chi):
    peaknum=len(peaklist)
    
    chisq=0.0
    for i in range(0,peaknum):
        pol,az=XYtoPOLAZ(peaklist[i][0],peaklist[i][1])
#        phi=peaklist[i][2]
        phi=peaklist[i][2]*phistepsize
        IN=hkl.Calc_HKL(np.asarray([pol]),np.asarray([az]),eta,mu,chi,phi,wl,UBR)
        dvec=IN-np.rint(IN) ### RINT rounds to integers, IN - RINT gives the difference between the UB calculated from the peak and a UB matrix whose elements are integers. 
        chisq=chisq+np.linalg.norm(dvec) ## norm just adds all the elements pretty much, if that sum is far from zero then the UB matrix picked is bad, and chisq is large. 
        
    return chisq/peaknum

## print the HKL values of peaks in peaklist for a given UB matrix and experimental parameters
def print_HKL(peaklist,wl,UBR,eta,mu,chi):
    peaknum=len(peaklist)
    
    chisq=0.0
    for i in range(0,peaknum):
        pol,az=XYtoPOLAZ(peaklist[i][0],peaklist[i][1])
#        phi=peaklist[i][2]
        phi=peaklist[i][2]*phistepsize
        IN=hkl.Calc_HKL(np.asarray([pol]),np.asarray([az]),eta,mu,chi,phi,wl,UBR)
        print(IN)
    
### minimization functions
def fun_min(x):
    UBR = calcUB(x[0],x[1], x[2], uca, ucb, ucc, ucal, ucbe, ucga)
    return chisq_peaklist(peaklist, wl, UBR.T, eta, mu, chi)


def minfunc(eu,a,b,c,alpha,beta,gamma,peaklist,wl):
    UBR=calcUB(eu[0],eu[1],eu[2],a,b,c,alpha,beta,gamma)
    chisq=chisq_peaklist(peaklist,wl,UBR.T,eta,mu,chi)
    return chisq



In [ ]:
### import modules, some from hklconv_new
import xrayutilities as xu
from PIL import Image
import numpy as np
import ctypes as ct
import os
import sys
import signal
import time
from nexusformat.nexus import *
import hkl
from pylab import *
from scipy import *
import tqdm
import fabio

from scipy.spatial.transform import Rotation as R
from scipy.optimize import minimize

In [ ]:
### scan parameters from the stack file

stack_file = "/data2/2023/chess_mar23/SiP/stacks/SiP_5.nxs"
stack=nxload(stack_file)

wl=stack.geo.wl*1e10#fix for old style fastsweep staks

eta=stack.psic.eta
mu=stack.psic.mu
chi=stack.psic.chi
#dpsi=0.09*np.pi/180.0
dpsi=0.0*np.pi/180.0
phi=stack.data.phi.nxdata #### note this is because only every 10th image was copied over
phistepsize = np.abs(phi[1]-phi[0])


myaz=np.asarray(stack.geo.az.data)
mypol=np.asarray(stack.geo.pol.data)
mytth=np.sqrt((myaz*myaz) + (mypol*mypol))
mypsi=np.asarray(stack.geo.psi.data)
newpol=mytth*np.cos(mypsi+(np.pi*0.5)+dpsi)
newaz=-mytth*np.sin(mypsi+(np.pi*0.5)+dpsi)
newpol=mypol
newaz=myaz




In [ ]:
### Looking at the peak list obtained earlier.
### Identifying Unique peaks
peaklist = np.load("/data2/2023/chess_mar23/SiP/peaklists/SiP-5", allow_pickle=True)

poni = pyFAI.load("/home/jg2364/chess_mar2023/calibration/lab6_26kev_new_svn.poni")
qmap = poni.qArray()


threshold = 1e4
peaklist_new =peaklist[np.where(peaklist[:,3] >threshold)]
print("total number of peaks")
print(len(peaklist_new))

peaks_unique = np.array([peaklist_new[0]])
    


for uu in range(0, len(peaklist_new), 1):
    #print(goodpeaks_unique)
    #print(len(goodpeaks_unique))
    for ww in range(len(peaks_unique)-1, len(peaks_unique), 1):
        #print(goodpeaks)
        boolean1 =  np.abs(peaklist_new[:,1][uu] - peaks_unique[:,1][ww]) <20
        boolean2 =  np.abs(peaklist_new[:,2][uu] - peaks_unique[:,2][ww]) <20
        #print(goodpeaks_unique)
        #print(boolean1, boolean2)
        
        if((boolean1==False) and (boolean2 == False)):
            peaks_unique = np.append(peaks_unique, np.array(peaklist_new[uu]))
            peaks_unique = np.reshape(peaks_unique, (-1, 4))
            break
        #if(ww==len(goodpeaks_unique)):
            #break
        
print("Total number of unique peaks: "+str(len(peaks_unique)))




peaklist = peaklist_new
#print(len(peaklist))
## Enter crystal parameters here
#system = xu.materials.Crystal("system", xu.materials.SGLattice(167, 4.76, 13.12))
system = xu.materials.Crystal("system", xu.materials.SGLattice(227, 5.44))
#https://materialsproject.org/materials/mp-149/

q_peaks = np.zeros(len(peaklist))
for ii in range(len(peaklist)):
    q_peaks[ii] = qmap[peaklist[ii][0]][peaklist[ii][1]]


uca=system.lattice.a
ucb=system.lattice.b
ucc=system.lattice.c
ucal=system.lattice.alpha
ucbe=system.lattice.beta
ucga=system.lattice.gamma


peaklist = np.column_stack((peaklist, q_peaks))

In [ ]:
# sort the peaklist
peaklist = peaklist[peaklist[:,4].argsort()] 

In [ ]:
## plot the powderized peaks
%matplotlib notebook
plt.plot(peaklist[:,4], peaklist[:,3], '-o')
plt.xlim(15, 55)
plt.ylim(0, .3e6)
#plt.ylim(.95e6, 1.15e6)

In [ ]:
## check the peak positions from the CIF
h_k_l = (2,2,4)
q_exp = system.Q(h_k_l)
print(10*norm(q_exp))
#113, 111, 222

In [ ]:
### pick peaks which are closest to the positions of the CIF

the_good = np.array([19.88, 32.69, 38.339, 39.953]) #the_good = np.array([20.005, 38.333, 40.01])
the_bad = np.zeros(len(the_good))

for jj in range(0, len(the_good), 1):
    the_bad[jj] = find_nearest(peaklist[:,4], the_good[jj])

the_bad = the_bad.astype(int)
the_ugly = peaklist[the_bad]
print(the_ugly)

In [ ]:
peaklist = the_ugly[:,:3]
peaklist = peaklist.astype(int)

In [ ]:
### set up angle range to solve for minimum UB matrix
phii = np.linspace(np.deg2rad(0), np.deg2rad(180), 15)
thetaa =  np.linspace(np.deg2rad(0), np.deg2rad(181),15)
chii =  np.linspace(np.deg2rad(0),np.deg2rad(90), 5)

In [ ]:
# set up bounds
bnds = ((phii[0]-np.deg2rad(8), phii[-1]+np.deg2rad(8)), (thetaa[0]-np.deg2rad(8), thetaa[-1]+np.deg2rad(8)), (chii[0]-np.deg2rad(8), chii[-1]+np.deg2rad(8)))

In [ ]:
### run the minimizer to find the UB matrix with the smallest chisquare in the given angle range
print("running the minimizer. may throw some exceptions which you can ignore. Be patient, a '.' is printed every 100 completed steps.")
#EulerAngles = np.zeros((len(phii)*len(thetaa)*len(chii), 3))
EulerMin = []
counter =0
chisq_brute_min = 1.0
for i in range(0, len(phii), 1):
    for j in range(0, len(thetaa), 1):
        for k in range(0, len(chii), 1):
            #print(phii[i], thetaa[j], chii[k])
            res = minimize(fun_min, (phii[i], thetaa[j], chii[k]), method='SLSQP', bounds=bnds)
            #print(res.fun)
            if(res.fun<chisq_brute_min):
                EulerMin = [res.x[0], res.x[1], res.x[2]]
                chisq_brute_min = res.fun
            if(counter%100 ==0):
                print('.', end='')
            counter = counter +1
print('\n')

print("The minimum ChiSq is: "+str(chisq_brute_min)+" at Euler Angles "+str(np.rad2deg(EulerMin[0]))+ ' , '+str(np.rad2deg(EulerMin[1]))+' , '+str(np.rad2deg(EulerMin[2])))
UBR = calcUB(EulerMin[0], EulerMin[1], EulerMin[2],uca, ucb, ucc, ucal, ucbe, ucga)
print("HKL of peaks of minimum UBmatrix: "+str(print_HKL(peaklist,wl,UBR.T,eta,mu,chi)))
    

In [ ]:
print("The minimum ChiSq is: "+str(chisq_brute_min)+" at Euler Angles "+str(np.rad2deg(EulerMin[0]))+ ' , '+str(np.rad2deg(EulerMin[1]))+' , '+str(np.rad2deg(EulerMin[2])))
UBR = calcUB(EulerMin[0], EulerMin[1], EulerMin[2],uca, ucb, ucc, ucal, ucbe, ucga)
print("HKL of peaks of minimum UBmatrix: "+str(print_HKL(peaklist,wl,UBR.T,eta,mu,chi)))
  

In [ ]:
### run an altnerative basin hopping minimizer 
from scipy.optimize import basinhopping
bnds = ((-np.pi,np.pi), (-np.pi, np.pi), (-np.pi, np.pi))
minimizer_kwargs = {"method": "L-BFGS-B", "args": (uca, ucb, ucc, ucal, ucbe, ucga, peaklist, wl), "bounds":bnds}
iterations = 300
    
### initial guess of the Euler angles. These are random. It then iterates through all the peaks and calculates the chisq. 


### niter is number of guesses. seed is the randomness start parameter. stepsize dictates the size of jumps in the random Euler angles. T determines what size of jumps in chisq will be accepted. 

stepsize =np.pi/20. #np.pi/30.
accept = 0.05 #0.05
x0 = (EulerMin[0], EulerMin[1], EulerMin[2])

ret1 = basinhopping(minfunc, x0, stepsize=stepsize, T=accept, minimizer_kwargs=minimizer_kwargs,  niter=iterations, seed=30)
print([ret1.x[0],ret1.x[1],ret1.x[2]])

In [ ]:
print("The minimum ChiSq is: "+str(ret1.fun)+" at Euler Angles "+str(np.rad2deg(ret1.x[0]))+ ' , '+str(np.rad2deg(ret1.x[1]))+' , '+str(np.rad2deg(ret1.x[2])))
UBR = calcUB(ret1.x[0], ret1.x[1], ret1.x[2],uca, ucb, ucc, ucal, ucbe, ucga)
print("HKL of peaks of minimum UBmatrix: "+str(print_HKL(peaklist,wl,UBR.T,eta,mu,chi)))

In [ ]:
chisq_peaklist(peaklist,wl,UBR.T,eta,mu,chi)

In [ ]:
### USE this UB matrix in the indexing code.
print(UBR.T)

[[ 0.832761  0.800328 -0.001698]
 [ 0.800132 -0.832608 -0.023912]
 [-0.017793  0.016064 -1.154748]]
